# Experiment 2D KP

### Load Model & Dataset

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from src.models.pose_resnet import BasicBlock, PoseResNet, Bottleneck
import src.bug_dataset
from src.eval.loss import JointsMSELoss
from src.eval.accuracies import accuracy
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
file_path = '../data/single_sungaya/label_names.txt'

with open(file_path) as file:
    label_names = file.readlines()
    label_names = [item.rstrip() for item in label_names]

In [3]:
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))
reduceKeypoints = True
sungaya_dataset = src.bug_dataset.BugDataset(df=out_df,
                             root_dir=target_dir, reduced=reduceKeypoints, transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))
train_split = 0.7
valid_split = 0.1
train_size = int(train_split * len(sungaya_dataset))
valid_size = int(valid_split * len(sungaya_dataset))
test_size = len(sungaya_dataset) - (train_size+valid_size)
_,_ ,test_dataset = torch.utils.data.random_split(sungaya_dataset, [train_size, valid_size, test_size], generator=torch.Generator().manual_seed(42))

batch_size = 128
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))   

# Create Model
num_layers = 18
resnet_spec = {18: (BasicBlock, [2, 2, 2, 2]),
               34: (BasicBlock, [3, 4, 6, 3]),
               50: (Bottleneck, [3, 4, 6, 3]),
               101: (Bottleneck, [3, 4, 23, 3]),
               152: (Bottleneck, [3, 8, 36, 3])}
if reduceKeypoints:
    block_class, layers = resnet_spec[num_layers]
    model = PoseResNet(block_class, layers, 28)

else:
    block_class, layers = resnet_spec[num_layers]
    model = PoseResNet(block_class, layers, 62)


criterion = JointsMSELoss(use_target_weight=True)
model = model.to(device)
path = "../models/2d_horiz_vertical_flip/simple_2d_model_20220321-195443_128"
model.load_state_dict(torch.load(path))
model.eval()
trained_model = model

Using cuda device


In [5]:
def test_func(model,ds, thr=0.5):
    test_epoch_acc, test_epoch_loss = 0, 0
    test_epoch_acc_kps = [0]*28
    batches = 0
    # Code get the first batch of results
    with torch.no_grad():
        for data in tqdm(ds, desc="2D Keypoint Experiment"):
            input = data['image'].to(device, dtype=torch.float)
            target_heatmap = data['heatmap'].to(device, dtype=torch.float)
            heat_weight = data['heat_weight'].to(device, dtype=torch.float)

            pred_heatmap =  model(input)

            test_loss = criterion(pred_heatmap, target_heatmap, heat_weight)

            test_acc_kps, test_acc, _, _ = accuracy(pred_heatmap.detach().cpu().numpy(),
                                            target_heatmap.detach().cpu().numpy(),thr=  thr)
                                            
            test_epoch_loss += test_loss.item()
            test_epoch_acc += test_acc
            test_epoch_acc_kps += test_acc_kps[1:29]
            batches +=1
            
    test_acc = test_epoch_acc/batches
    test_acc_kps = test_epoch_acc_kps/batches
    test_loss = test_epoch_loss/batches
    return test_acc, test_loss, test_acc_kps


In [6]:
trained_model.eval()
test_acc, test_loss, test_acc_kps = test_func(trained_model, test_dataloader)
print(test_acc)
print(test_loss)

2D Keypoint Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

0.555750633454075
0.002602711746779581


In [7]:
# Plots the accuracies per KP at normal threshold
 
fig, ax = plt.subplots()

ax.barh(np.array(label_names)[sungaya_dataset.reduced_kp], test_acc_kps, linewidth =5, edgecolor="white")
ax.invert_yaxis()
# ax.set(xlim=(-1, 8), xticks=np.arange(-1, 8),
#        ylim=(0, 1), yticks=np.arange(0, 0.5))
plt.title("2D PCK Accuracies for every Keypoint")

plt.xlabel("Accuracies")
plt.ylabel("Keypoints")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# Need to do an experiment where we vary the threshold value on the test set
thresholds = [0.01 ,0.1, 0.5, 1.0, 5.0, 10.0, 20.0] # could use np.linspace if need be
# thresholds = np.linspace(0.1,5,8)
accs = []
losses = []
acc_kps = []
for thr in thresholds:
    test_acc, test_loss, test_acc_kps = test_func(trained_model, test_dataloader, thr)
    accs.append(test_acc)
    losses.append(test_loss)
    acc_kps.append(test_acc_kps)

2D Keypoint Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
# plots PCK for different thresholds
fig, ax = plt.subplots()

ax.bar(list(map(str,thresholds)), accs, width=1, edgecolor="white", linewidth=5)

# ax.set(xlim=(-1, 8), xticks=np.arange(-1, 8),
#        ylim=(0, 1), yticks=np.arange(0, 0.5))
plt.title("2D PCK Accuracies at Varying Thresholds")
plt.xlabel("Thresholds")
plt.ylabel("Accuracies")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …